<a href="https://colab.research.google.com/github/yujiimt/recsys/blob/master/recsys2019/recsys_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
import gc
import re
from tqdm import tqdm
import copy
import os
import pickle

PATH = "/content/drive/My Drive/trivagoRecSysChallengeData2019_v2/"

In [5]:
meta_df = pd.read_csv(PATH + "item_metadata.csv")
print(len(meta_df))
print("-" * 10)
print(meta_df.isnull().sum())
meta_df.head()

927142
----------
item_id       0
properties    0
dtype: int64


,item_id,properties
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
2,5834,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
3,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone...
4,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Sa...


In [6]:
train_df = pd.read_csv(PATH + "train.csv")
print(len(train_df))
print("-" * 10)
print(train_df.isnull().sum())
train_df.head()

15932992
----------
user_id                   0
session_id                0
timestamp                 0
step                      0
action_type               0
reference                 0
platform                  0
city                      0
device                    0
current_filters    14779880
impressions        14346406
prices             14346406
dtype: int64


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [7]:
test_df = pd.read_csv(PATH + "test.csv")
print(len(test_df))
print("-" * 10)
print(test_df.isnull().sum())
test_df.head()

3782335
----------
user_id                  0
session_id               0
timestamp                0
step                     0
action_type              0
reference           261731
platform                 0
city                     0
device                   0
current_filters    3439491
impressions        3253556
prices             3253556
dtype: int64


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
3,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
4,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...


In [6]:
sub = pd.read_csv(PATH + "submission_popular.csv")
print(len(sub))
sub.head()

253573


,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,924795 106315 1033140 119494 101758 903037 105...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,3505150 3812004 2227896 2292254 3184842 222702...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,4476010 3505150 3812004 2227896 2292254 222702...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22854 3067559 22721 22713 16121 22772 22727 22...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,9857656 5849628 655716 1352530 502066 1405084 ...


In [8]:
def unroll(t):
  idx = []
  suffix = []
  to_append_partial = []

  a_user = t.loc[0].user_id
  a_sess = t.loc[0].session_id
  count = -1
  to_append = []
  for r in tqdm(zip(t.user_id, t.session_id, t.action_type)):
    count += 1
    if r[0] == a_user and r[1] == a_sess:
      to_append.append(count)
      if r[2] == "clickout item":
        to_append_partial += copy.deepcopy([to_append])
      else:
        for i in range(len(to_append_partial) -1):
          idx += to_append_partial[i]
          suffix += ["_"*(i+1) for j in to_append_partial[i]]
        idx += to_append
        suffix += ["" for j in to_append]
        a_user = t.loc[count].user_id
        a_sess = t.loc[count].session_id
        to_append = [count]
        to_append_partial = []
        if r[2] == 'clickout item':
          to_append_partial += copy.deepcopy([to_append])
  for i in range(len(to_append_partial)-1):
      idx += to_append_partial[i]
      suffix += ["_"*(i+1) for j in to_append_partial[i]]
      
  t_new = t.loc[idx]
  user_with_suffix = []
  sess_with_suffix = []
  count = 0
  for r in tqdm(zip(t_new.user_id, t_new.session_id)):
    user_with_suffix.append("{}{}".format(r[0], suffix[count]))
    sess_with_suffix.append("{}{}".format(r[1], suffix[count]))
    count += 1
  t_new["user_id"] = user_with_suffix
  t_new["session_id"] = sess_with_suffix
  return t_new

In [9]:
unroll(train_df)

15932992it [00:08, 1915636.32it/s]
28it [00:00, 44518.77it/s]


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [12]:
def reset_step_for(df):
  res_df = df.copy()
  df_dup = df[["session_id", "user_id", "step"]]
  df_dup = df_dup[df_dup["step"] == 1]
  df_dup = df_dup.groupby(["user_id", "session_id", "step"]).size() \
    .sort_values(ascending=False) \
    .reset_index(name="count")
  df_dup = df_dup[df_dup["count"] > 1]
  df_dup = df_dup[["user_id", "session_id"]]

  for _, row in tqdm(df_dup.iterrows()):
    mask = (df.user_id == row.user_id) & (df.session_id == row.session_id)
    sess_length = sum(mask*1)
    res_df.loc[mask, "step"] = np.arange(1, sess_length+1, dtype="int")

  return res_df 

In [ ]:
reset_step_for(train_df)

In [14]:
train_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [29]:
def merge_duplicates(df):
    tqdm.pandas()

    duplicates_indices = []
    indices = df.index.values
    totlen = len(df)
    i = 0
    j = 0
    next_index = indices[j]

    for index in tqdm(indices):
        if i >= j:
            curr_actiontype = df.at[index,'action_type']
            count = 1
            j += 1
            while j < totlen:
                next_index = indices[j]

                if curr_actiontype != 'clickout item' and \
                    df.at[index, 'user_id'] == df.at[next_index, 'user_id'] and \
                    df.at[index, 'session_id'] == df.at[next_index, 'session_id'] and \
                    df.at[index, 'reference'] == df.at[next_index, 'reference'] and \
                    curr_actiontype == df.at[next_index, 'action_type']:

                    j += 1
                    duplicates_indices.append(next_index)
                    count += 1
                else:
                    break

            df.at[index, 'frequence'] = count
        i += 1

    return df.drop(duplicates_indices)

In [30]:
merge_duplicates(train_df)

100%|██████████| 15932992/15932992 [16:26<00:00, 16144.94it/s]


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15932974,ZYNMLE3MV3LK,62728015bec05,1541544470,2,interaction item image,6617798,PT,"Paris, France",desktop,NaN,NaN,NaN,14.0
15932988,ZYNMLE3MV3LK,62728015bec05,1541544491,16,clickout item,6617798,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...,1.0
15932989,ZYNMLE3MV3LK,62728015bec05,1541544540,17,clickout item,2712342,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...,1.0
15932990,ZYNMLE3MV3LK,62728015bec05,1541544967,18,change of sort order,interaction sort button,PT,"Paris, France",desktop,NaN,NaN,NaN,1.0
